<a href="https://colab.research.google.com/github/Midas0901/Poom/blob/main/IAT360_TOS_RAG_Final_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q sentence-transformers faiss-cpu langchain langchain-community langchain-text-splitters --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
# 2. IMPORTS
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
import json
from tqdm import tqdm

In [16]:
# ========================================================
# FINAL COMPLETE CELL – ALL 4 DATASETS (Google Colab 2025)
# ========================================================
print("=" * 70)
print("LOADING ALL 4 DATASETS ")
print("=" * 70)

from datasets import load_dataset, Dataset
import pandas as pd
import glob
import os

# --------------------------------------------------------------
# 1. OPP-115 Dataset
print("\n1. Downloading OPP-115 dataset...")
dataset_1 = load_dataset("alzoubi36/opp_115")

# 2. PrivacyPolicy Dataset
print("2. Downloading PrivacyPolicy dataset...")
dataset_2 = load_dataset("sjsq/PrivacyPolicy")

# 3. YOUR OWN DATA – Own data.txt
print("3. Loading your custom 'Own data.txt'...")
if not os.path.exists("/content/Own data.txt"):
    from google.colab import files
    print("Please upload 'Own data.txt' now...")
    uploaded = files.upload()

with open("/content/Own data.txt", "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip() and not line.startswith("#")]

data3 = []
for line in lines:
    if len(line) >= 2 and line[-2] == " " and line[-1].isdigit():
        text = line[:-2].rstrip()
        label = int(line[-1])
        data3.append({"text": text, "label": label})

dataset_3 = Dataset.from_pandas(pd.DataFrame(data3))
dataset_3.class_names = [
    "Forced Monthly Payments",
    "Data Collection & Sharing with third parties",
    "Unfair Control Over Your Content",
    "No Refund Policy",
    "Good and safe"
]

# --------------------------------------------------------------
# 4. FULL TOSDR Corpus – ALL 9,491 documents (via kagglehub)
print("\n4. Downloading FULL TOSDR Corpus (real Terms of Service)...")

# Install & use kagglehub (fast & reliable)
!pip install -q kagglehub --upgrade

import kagglehub

# Download once (cached forever in Colab)
path = kagglehub.dataset_download("sonu1607/tosdr-terms-of-service-corpus")
print("Dataset files at:", path)

# Find all .txt files
txt_files = glob.glob(f"{path}/**/*.txt", recursive=True)
print(f"   Found {len(txt_files):,} TOS documents")

# Load ALL documents (with progress)
print("   Loading all documents into dataset_4 (this may take 1–2 minutes)...")
data4 = []

for i, file_path in enumerate(txt_files):
    try:
        with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
            text = f.read().strip()
            if len(text) > 200:  # skip tiny or empty files
                data4.append({"text": text, "label": -1})
        if (i + 1) % 1000 == 0:
            print(f"      Loaded {i+1:,}/{len(txt_files):,} documents...")
    except:
        continue  # skip corrupted files

dataset_4 = Dataset.from_pandas(pd.DataFrame(data4))

# --------------------------------------------------------------
print("\n" + "="*70)
print("SUCCESS! ALL 4 DATASETS LOADED")
print("="*70)
print(f"   • OPP-115               : {len(dataset_1['train']):,} labeled clauses")
print(f"   • PrivacyPolicy         : {len(dataset_2['train']):,} labeled clauses")
print(f"   • Your Custom Data      : {len(dataset_3):,} clauses")
print(f"   • TOSDR Full Corpus     : {len(dataset_4):,} full ToS documents")
print("="*70)
print("You now have one of the largest privacy/ToS datasets in the world!")
print("Ready for RAG, fine-tuning, or classification!")

LOADING ALL 4 DATASETS 

1. Downloading OPP-115 dataset...
2. Downloading PrivacyPolicy dataset...
3. Loading your custom 'Own data.txt'...

4. Downloading FULL TOSDR Corpus (real Terms of Service)...
Using Colab cache for faster access to the 'tosdr-terms-of-service-corpus' dataset.
Dataset files at: /kaggle/input/tosdr-terms-of-service-corpus
   Found 9,491 TOS documents
   Loading all documents into dataset_4 (this may take 1–2 minutes)...
      Loaded 1,000/9,491 documents...
      Loaded 2,000/9,491 documents...
      Loaded 3,000/9,491 documents...
      Loaded 4,000/9,491 documents...
      Loaded 5,000/9,491 documents...
      Loaded 6,000/9,491 documents...
      Loaded 7,000/9,491 documents...
      Loaded 8,000/9,491 documents...
      Loaded 9,000/9,491 documents...

SUCCESS! ALL 4 DATASETS LOADED
   • OPP-115               : 2,185 labeled clauses
   • PrivacyPolicy         : 1,789 labeled clauses
   • Your Custom Data      : 151 clauses
   • TOSDR Full Corpus     : 9,469 f

In [19]:
# 4. CONVERT ALL 4 DATASETS TO LANGCHAIN DOCUMENTS
print("\n" + "=" * 50)
print("CONVERTING ALL 4 DATASETS TO LANGCHAIN DOCUMENTS")
print("=" * 50)

from langchain_core.documents import Document
from tqdm import tqdm
import os

documents = []

# ===============================================
# Dataset 1: OPP-115
# ===============================================
print("\n1. Processing OPP-115 dataset...")
for split in ['train', 'validation', 'test']:
    if split in dataset_1:
        print(f"   → {split} ({len(dataset_1[split])} examples)")
        for i, item in enumerate(tqdm(dataset_1[split], desc=f"OPP-115 {split}")):
            documents.append(Document(
                page_content=item['text'],
                metadata={
                    'source': 'OPP-115',
                    'dataset': 'opp_115',
                    'split': split,
                    'index': i,
                    'doc_id': f"opp115_{split}_{i}",
                    'label': item.get('label', -1)
                }
            ))

# ===============================================
# Dataset 2: sjsq/PrivacyPolicy
# ===============================================
print("\n2. Processing sjsq/PrivacyPolicy dataset...")
for split in dataset_2.keys():
    print(f"   → {split} ({len(dataset_2[split])} examples)")
    for i, item in enumerate(tqdm(dataset_2[split], desc=f"PrivacyPolicy {split}")):
        text = item.get('Text') or item.get('text') or ""
        if text and str(text).lower() != "null":
            documents.append(Document(
                page_content=text,
                metadata={
                    'source': 'PrivacyPolicy Dataset',
                    'dataset': 'sjsq_privacy_policy',
                    'split': split,
                    'index': i,
                    'filename': item.get('filename', 'unknown'),
                    'doc_id': f"privpol_{split}_{i}"
                }
            ))

# ===============================================
# Dataset 3: Your Own Custom Data (Own data.txt)
# ===============================================
print("\n3. Processing Your Custom Data (Own data.txt)...")
print(f"   → {len(dataset_3)} labeled clauses")
for i, item in enumerate(tqdm(dataset_3, desc="Your Custom Data")):
    documents.append(Document(
        page_content=item['text'],
        metadata={
            'source': 'Your Custom Dataset',
            'dataset': 'your_own_data',
            'index': i,
            'label': item['label'],
            'label_name': dataset_3.class_names[item['label']] if item['label'] != -1 else "N/A",
            'doc_id': f"custom_{i}"
        }
    ))

# =============================================== =================================
# Dataset 4: FULL TOSDR Corpus (~9,491 real ToS documents)
# ===============================================
print("\n4. Processing FULL TOSDR Corpus (raw Terms of Service)...")
print(f"   → {len(dataset_4)} full policy documents (this may take 10-20 seconds)")

for i, item in enumerate(tqdm(dataset_4, desc="TOSDR Full Documents")):
    # Extract filename from path for better metadata
    file_path = txt_files[i] if i < len(txt_files) else "unknown"
    filename = os.path.basename(file_path)

    documents.append(Document(
        page_content=item['text'],
        metadata={
            'source': 'TOSDR Corpus',
            'dataset': 'tosdr_full',
            'index': i,
            'filename': filename,
            'doc_id': f"tosdr_{i}",
            'type': 'full_terms_of_service',
            'character_count': len(item['text'])
        }
    ))

# ===============================================
# FINAL SUMMARY
# ===============================================
print("\n" + "=" * 60)
print("ALL DOCUMENTS SUCCESSFULLY CONVERTED!")
print("=" * 60)
print(f"   Total LangChain Documents : {len(documents):,} ")
print(f"   • From OPP-115            : {sum(1 for d in documents if d.metadata['dataset'] == 'opp_115'):,}")
print(f"   • From PrivacyPolicy      : {sum(1 for d in documents if 'sjsq_privacy_policy' in d.metadata.get('dataset', '')):,}")
print(f"   • From Your Custom Data   : {len(dataset_3):,}")
print(f"   • From TOSDR Full Corpus  : {len(dataset_4):,}")
print("=" * 60)
print("Ready for embedding + FAISS vector store!")


CONVERTING ALL 4 DATASETS TO LANGCHAIN DOCUMENTS

1. Processing OPP-115 dataset...
   → train (2185 examples)


OPP-115 train: 100%|██████████| 2185/2185 [00:00<00:00, 7205.77it/s]


   → validation (550 examples)


OPP-115 validation: 100%|██████████| 550/550 [00:00<00:00, 14726.81it/s]


   → test (697 examples)


...


3. Processing Your Custom Data (Own data.txt)...
   → 151 labeled clauses


Your Custom Data: 100%|██████████| 151/151 [00:00<00:00, 25424.11it/s]



4. Processing FULL TOSDR Corpus (raw Terms of Service)...
   → 9469 full policy documents (this may take 10-20 seconds)


TOSDR Full Documents: 100%|██████████| 9469/9469 [00:00<00:00, 16392.95it/s]


ALL DOCUMENTS SUCCESSFULLY CONVERTED!
   Total LangChain Documents : 14,841 
   • From OPP-115            : 3,432
   • From PrivacyPolicy      : 1,789
   • From Your Custom Data   : 151
   • From TOSDR Full Corpus  : 9,469
Ready for embedding + FAISS vector store!


In [20]:
# 5. CHUNK THE DOCUMENTS
print("\n" + "=" * 30)
print("CHUNKING DOCUMENTS FOR RETRIEVAL")

print("\nSplitting text into searchable chunks...")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    separators=["\n\n", "\n", ". ", " "],
    length_function=len,
)

chunks = text_splitter.split_documents(documents)

print(f"\n✅ Created {len(chunks)} searchable chunks")
print(f"   Average chunk size: ~{sum(len(c.page_content) for c in chunks) // len(chunks)} characters")


CHUNKING DOCUMENTS FOR RETRIEVAL

Splitting text into searchable chunks...

✅ Created 343107 searchable chunks
   Average chunk size: ~579 characters


In [21]:
# 6. CREATE EMBEDDINGS MODEL
print("\n" + "=" * 30)
print("LOADING EMBEDDING MODEL FROM HUGGING FACE")

print("\nLoading: sentence-transformers/all-MiniLM-L6-v2")
print("   - Model size: 80MB")
print("   - Embedding dimensions: 384")
print("   - License: Apache 2.0 (free for all uses)")

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

print("\nEmbedding model loaded successfully!")


LOADING EMBEDDING MODEL FROM HUGGING FACE

Loading: sentence-transformers/all-MiniLM-L6-v2
   - Model size: 80MB
   - Embedding dimensions: 384
   - License: Apache 2.0 (free for all uses)


/tmp/ipython-input-2559379280.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


Embedding model loaded successfully!


In [ ]:
# 7. BUILD VECTOR DATABASE
print("\n" + "=" * 30)
print("BUILDING FAISS VECTOR DATABASE")

print("\n🔨 Creating embeddings for all chunks...")
print("   (This may take 2-3 minutes for 3,000+ chunks)")

vectorstore = FAISS.from_documents(chunks, embeddings)

print("\nVector database created successfully!")
print(f"   Total vectors: {vectorstore.index.ntotal}")



BUILDING FAISS VECTOR DATABASE

🔨 Creating embeddings for all chunks...
   (This may take 2-3 minutes for 3,000+ chunks)


In [ ]:
# 8. TEST THE DATABASE
print("\n" + "=" * 30)
print("TESTING RETRIEVAL")

test_queries = [
    "Can the company sell my personal data to third parties?",
    "Do they use my content to train AI models?",
    "Can I delete all my data from their servers?",
    "Am I forced into arbitration instead of court?"
]

print("\nRunning test queries...\n")

for query in test_queries:
    print(f"Query: '{query}'")
    results = vectorstore.similarity_search(query, k=2)
    print(f"Found {len(results)} relevant chunks")
    print(f"   Top result preview: {results[0].page_content[:150]}...")
    print()


TESTING RETRIEVAL

Running test queries...

Query: 'Can the company sell my personal data to third parties?'
Found 2 relevant chunks
   Top result preview: Note, that we will not share your Personally Identifiable Information with third parties for their marketing purposes without obtaining your prior con...

Query: 'Do they use my content to train AI models?'
Found 2 relevant chunks
   Top result preview: Our automated systems analyze your content (including emails) to provide you personally relevant product features, such as customized search results, ...

Query: 'Can I delete all my data from their servers?'
Found 2 relevant chunks
   Top result preview: Remember that even after you cancel your account, copies of some information from your account may remain viewable in some circumstances where, for ex...

Query: 'Am I forced into arbitration instead of court?'
Found 2 relevant chunks
   Top result preview: . Should you file any arbitration claims, or any administrative or legal ac

In [ ]:
# 9. SAVE THE DATABASE
print("=" * 30)
print("SAVING DATABASE")

print("\n💾 Saving FAISS database to disk...")
vectorstore.save_local("faiss_index_tos_hf")

print("Database saved to: faiss_index_tos_hf/")

SAVING DATABASE

💾 Saving FAISS database to disk...
Database saved to: faiss_index_tos_hf/


In [ ]:
# 10. CREATE METADATA FILE
metadata = {
    "dataset": "alzoubi36/opp_115",
    "total_documents": len(documents),
    "total_chunks": len(chunks),
    "embedding_model": "sentence-transformers/all-MiniLM-L6-v2",
    "embedding_dimension": 384,
    "chunk_size": 800,
    "chunk_overlap": 100,
    "vector_database": "FAISS",
    "date_created": "2024-12",
    "license": "Apache 2.0",
    "use_case": "Terms of Service Analysis - IAT 360"
}

with open('faiss_index_tos_hf/metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("Metadata saved")

Metadata saved


In [ ]:
# 11. ZIP FOR DOWNLOAD
print("\nCreating zip file for download...")
!zip -r -q faiss_index_tos_hf.zip faiss_index_tos_hf

print("\n" + "=" * 30)
print("SUCCESS! DATABASE READY FOR DEPLOYMENT")

# Display file sizes
print("\n📊 File sizes:")
!du -sh faiss_index_tos_hf
!du -sh faiss_index_tos_hf.zip


Creating zip file for download...

SUCCESS! DATABASE READY FOR DEPLOYMENT

📊 File sizes:
12M	faiss_index_tos_hf
9.1M	faiss_index_tos_hf.zip
